# **Welcome to the Notebook**

#  Installing pyspark module

In [4]:
!pip install pyspark

     |████████████████████████████████| 212.4 MB 63 kB/s 
     |████████████████████████████████| 198 kB 47.4 MB/s 
  Created wheel for pyspark: filename=pyspark-3.1.2-py2.py3-none-any.whl size=212880768 sha256=d805b9a267c75e791c472423cf390a9025a3a91baae4286473347ef33721a180
  Stored in directory: /root/.cache/pip/wheels/a5/0a/c1/9561f6fecb759579a7d863dcd846daaa95f598744e71b02c77
Successfully built pyspark


Importing the modules 

In [5]:
from pyspark.sql import SparkSession
from pyspark.sql.functions import count, desc , col, max , struct
import matplotlib.pyplot as plts

creating spark session

In [6]:
spark = SparkSession.builder.appName('spark_app').getOrCreate()

### importing the *Listenings.csv* file: 

In [1]:
listening_csv_path= 'dataset/listenings.csv'
listening_df= spark.read.format('csv').option('inferSchema',True).option('header',True).load(listening_csv_path)

NameError: name 'spark' is not defined

let's check the data:

In [ ]:
listening_df.show()

let's delete useless columns:

In [ ]:
listening_df=listening_df.drop('date')

drop the null rows:

In [ ]:
listening_df= listening_df.na.drop()

let's check the dataset again:

In [ ]:
listening_df.show()

let's see the schema: 

In [ ]:
listening_df.printSchema()

let's see the shape of our dataframe: 

In [ ]:
shape=(listening_df.count(),len(listening_df.columns))
print(shape)

### select two columns: track and artist

In [ ]:
q0=listening_df.select('artist','track')
q0.show()

**Query #1**:

Let's find all of the records of those users who have listened to ***Rihanna***

In [ ]:
q1=listening_df.select('*').filter(listening_df.artist=='Rihanna')
q1.show()

**Query #2:**

Let's find top 10 users who are fan of ***Rihanna***

In [ ]:
q2=listening_df.select('user_id').filter(listening_df.artist=='Rihanna').groupby('user_id').agg(count('user_id').alias('count')).orderBy(desc('count')).limit(10)
q2.show()

**Query #3:**

find top 10 famous tracks 

In [ ]:
q3=listening_df.select('artist','track').groupby('artist','track').agg(count('*').alias('count')).orderBy(desc('count')).limit(10)
q3.show()

**Query #4:**

find top 10 famous tracks of ***Rihanna*** 

In [ ]:
q4=listening_df.select('artist','track').filter(listening_df.artist=='Rihanna').groupby('artist','track').agg(count('*').alias('count')).orderBy(desc('count')).limit(10)
q4.show()

**Query #5:**

find top 10 famous albums 

In [ ]:
q5=listening_df.select('artist','album').groupby('artist','album').agg(count('*').alias('count')).orderBy(desc('count')).limit(10)
q5.show()

### importing the ***genre.csv*** file:

In [ ]:
genre_csv_path='/content/drive/MyDrive/dataset/genre.csv'
genre_df = spark.read.format('csv').option('inferSchema',True).option('header',True).load(genre_csv_path)

let's check the data

In [ ]:
genre_df.show()

Let's inner join these two data frames

In [3]:
data=listening_df.join(genre_df,how="inner",on=['artist'])
data.show()

NameError: ignored

**Query #6**

find top 10 users who are fan of ***pop*** music

In [ ]:
q6=data.select('user_id').filter(data.genre=='pop').groupby('user_id').agg(count('*').alias('count')).orderBy(desc('count')).limit(10)
q6.show()

**Query #7**

find top 10 famous genres

In [ ]:
q7=data.select('genre').groupby('genre').agg(count('*').alias('count')).orderBy(desc('count')).limit(10)

q7.show()

### **Query #8**

find out each user favourite genre

In [ ]:
q8_1=data.select('user_id','genre').groupby('user_id','genre').agg(count('*').alias('count')).orderBy('user_id')
q8_1.show()

In [ ]:
q8_2=q8_1.groupby('user_id').agg(max(struct(col('count'),col('genre'))).alias('max')).select(col('user_id'),col('max.genre'))
q8_2.show()